In [1]:
import os
os.chdir("T:/laupodteam/AIOS/Chontira/CellDynClustering")
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score
import pandas as pd
import hdbscan
from evaluation.fast_dunn_index import dunn_fast
from data.load_data import *
import ast
from graphic_stuffs import *
import pickle
from evaluation.hyperparameter_optimization import *





## Evaluators

evaluators= {'silhouette_score': silhouette_score,
            'davies_bouldin_score': davies_bouldin_score,
            'dunn_fast': dunn_fast}

## data loading
dm6_with_labels = pd.read_feather("data/embedded_celldyn_ALL_nn50_ndim6_w_labels_.feather")
others_labels = dm6_with_labels.loc[:,dm6_with_labels.columns.isin(['sex', 'age','study_id', 'analysis_dt', 'sample_dt'])]
dm6 = load_data().set_df(dm6_with_labels.iloc[:, 0:6]).get_x_from_df(contain_y=False)

##order of sorting results
ascending = [False,True, False]


In [2]:
## Paramter set for tuning of HDBSCAN with outliers
f = open('models/hdbscan_param_grid.pickle', 'rb')
param_grid_full = pickle.load(f)
f.close()
param_grid_full

{'min_samples': [5, 10, 15, 20, 25],
 'min_cluster_size': [2, 5, 10, 20],
 'cluster_selection_epsilon': [0, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0],
 'cluster_selection_method': ('eom', 'leaf'),
 'metric': ('euclidean', 'manhattan', 'mahalanobis')}

In [3]:
## Tuning of HDBSCAN with outliers
results = random_seach_optimization(dm6.X,hdbscan.HDBSCAN,evaluators=evaluators,param_grid=param_grid_full,ascending=ascending, seperating_param_values=True,subsampling=20000, max_evals=450, num_iter=10)
#results.to_csv("models/cell_dyn_hdbscan_hyper_op_outliers_results_20000.csv")

Around 10% done.
Around 20% done.
Around 30% done.
Around 40% done.
Around 50% done.
Around 60% done.
Around 70% done.
Around 80% done.
Around 90% done.
Hyperpameter tuning is done and the best scores are:
silhouette_score        0.282535
davies_bouldin_score     2.87282
dunn_fast               0.009253
Name: 0, dtype: object
Number of unique labels 3
with parameter: {'min_samples': 15, 'min_cluster_size': 5, 'cluster_selection_epsilon': 1.0, 'cluster_selection_method': 'eom', 'metric': 'euclidean'}
Finish tuning in  0.35 minutes.


In [7]:
results_csv = pd.read_csv("models/cell_dyn_hdbscan_hyper_op_outliers_results_20000.csv")
                                                                                           